In [27]:
import pandas as pd
from sqlalchemy import Index, create_engine
import requests
import _local_settings
import datetime
import time
from sqlalchemy.types import Integer, TEXT, String, DateTime, NVARCHAR, Boolean, Float


ALL_APPS_URL = "http://api.steampowered.com/ISteamApps/GetAppList/v0002/?key=F4BD90D47EEA3DF3798A9A7E84AFF91C&format=json"
config = {
    'host': 'localhost',
    'port': 3306,
    'user': 'newuser',
    'password': 'newpassword',
    'database': 'test_db'
}

def open_db_connection():
    # specify database configurations
    db_user = config.get('user')
    db_pwd = config.get('password')
    db_host = config.get('host')
    db_port = config.get('port')
    db_name = config.get('database')
    # specify connection string
    connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'
    # connect to database
    engine = create_engine(connection_str)
    connection = engine.connect()

    return engine, connection


def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.

    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request

    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except requests.exceptions.Timeout:
        for i in range(5, 0, -1):
            print('\rTimeout. Waiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' ' * 10)
        return get_request(url, parameters)
    except requests.exceptions.TooManyRedirects:
        print('TooManyRedirects')
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        raise SystemExit(e)

    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

In [3]:
from ast import literal_eval

def load_data():
    def try_literal_eval(s):
        try:
            return literal_eval(s)
        except ValueError:
            return s


    steam_data = pd.read_csv('initial_preprocessed.csv') 
    # steam_data = pd.json_normalize(steam_data, errors='ignore')
    # all columns that are dicts are being read in as strings - look in to json_normalize as possibly better solution?
    steam_data['recommendations'] = steam_data.recommendations.apply(try_literal_eval)
    steam_data['screenshots'] = steam_data.screenshots.apply(try_literal_eval)
    steam_data['movies'] = steam_data.movies.apply(try_literal_eval)
    steam_data['genres'] = steam_data.genres.apply(try_literal_eval)
    steam_data['release_date'] = steam_data.release_date.apply(try_literal_eval)
    steam_data['fullgame'] = steam_data.fullgame.apply(try_literal_eval)
    steam_data['demos'] = steam_data.demos.apply(try_literal_eval)
    steam_data['categories'] = steam_data.categories.apply(try_literal_eval)
    steam_data['metacritic'] = steam_data.metacritic.apply(try_literal_eval)
    steam_data['achievements'] = steam_data.achievements.apply(try_literal_eval)

    return steam_data

df = load_data()

In [4]:
def flatten_field(df, field, rename_dict, drops_list):
    '''
    takes in a dataframe column that is a dict and separates it into
    separate columns per key/value pair.  Can rename cols and drop
    columns as specified

    df: dataframe to alter
    field: column to flatten
    rename_dict: dictionary of current_name: new_name pairs for updating
    drops_list: list of new columns to drop
    '''
    df_clean = pd.concat([df, df[field].apply(pd.Series)], axis=1)
    df_clean.drop(axis=1, columns=drops_list, inplace=True)
    df_clean.rename(columns=rename_dict, inplace=True)
    

    return df_clean

In [5]:
df_clean = flatten_field(df,
                         'fullgame',
                                 {'appid': 'fullgame_appid'},
                                 ['name', 'fullgame'])
print(df_clean['fullgame_appid'])


0         NaN
1      215280
2         NaN
3      544970
4      575170
        ...  
938    551730
939    551730
940    551730
941    551730
942    551730
Name: fullgame_appid, Length: 943, dtype: object


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   index                    943 non-null    int64 
 1   success                  943 non-null    bool  
 2   type                     943 non-null    object
 3   name                     943 non-null    object
 4   steam_appid              943 non-null    int64 
 5   required_age             943 non-null    int64 
 6   is_free                  943 non-null    bool  
 7   detailed_description     745 non-null    object
 8   about_the_game           745 non-null    object
 9   short_description        919 non-null    object
 10  supported_languages      939 non-null    object
 11  header_image             943 non-null    object
 12  website                  527 non-null    object
 13  pc_requirements          943 non-null    object
 14  mac_requirements         943 non-null    o

In [7]:
def create_unique_bool_cols(df, col, table):
    '''
    Takes in a single columns in a dataframe, detmerines all unique values,
    creates a column for each unique value in the dataframe and fills it
    with a bool for each row indicating if that values exists for that row

    assumes column splits out into ['id','description'] pairs for uniqueness

    new column names will all be delimited with underscore

    df: Dataframe
    col: column to split out into multiple bool columns
    prefix: prefix of the new column names (genre -> genre_action, genre_adventure...).
    Will use description to build new column name
    '''
    # first we need to create a table of all possible values then store those so we can access them
    # combine everything into single column

    # todo: create new df with steam_appid as index then fill in teh cool columns
#     new_df = pd.DataFrame(df[col], df['steam_appid'])
    new_df = df[['steam_appid', col]].copy()
    
    print(new_df)
    
    s = new_df[col].apply(pd.Series)
    num_cols = s.shape[1]

    # from pandas docs: Iteratively appending to a Series can be more computationally intensive than a single
    # concatenate. A better solution is to append values to a list and then concatenate the list with the original
    # Series all at once.
    listified = s[0].tolist()
    for x in range(1, num_cols):
        sub_list = s[x].dropna().tolist()
        listified += sub_list

    y = pd.Series(listified)

    y = y.dropna()

    # pull the dict out to columns
    z = y.apply(pd.Series)
    z = z.drop_duplicates(keep="first")

    # create a new column for each unique value
    for index, row in z.iterrows():
        # new_col = '{0} {1}'.format(prefix, row['description']).replace(" ", "_")
        # df[new_col] = False
        new_df[row['description']] = False

    # then fill those columns in the Dataframe with bools
    for index, row in new_df.iterrows():
        if type(row[col]) == float:
            continue
        for item in row[col]:
            # new_name = '{0} {1}'.format(prefix, item['description']).replace(" ", "_")
            # # because you can't update on  iterrows()
            # df.at[index, new_name] = True
            new_df.at[index, item['description']] = True

    # drop the original column at the end of processing
    new_df.drop(axis=1, columns=col, inplace=True)

    return new_df

In [8]:
df = load_data()
df_clean = create_unique_bool_cols(df, 'genres', 'genres')
df_clean

     steam_appid                                             genres
0         579920  [{'id': '53', 'description': 'Design & Illustr...
1         579930  [{'id': '1', 'description': 'Action'}, {'id': ...
2         579940           [{'id': '2', 'description': 'Strategy'}]
3         579960  [{'id': '25', 'description': 'Adventure'}, {'i...
4         579970  [{'id': '1', 'description': 'Action'}, {'id': ...
..           ...                                                ...
938       569685             [{'id': '1', 'description': 'Action'}]
939       569686             [{'id': '1', 'description': 'Action'}]
940       569687             [{'id': '1', 'description': 'Action'}]
941       569688             [{'id': '1', 'description': 'Action'}]
942       569689             [{'id': '1', 'description': 'Action'}]

[943 rows x 2 columns]


,steam_appid,Design & Illustration,Action,Strategy,Adventure,Indie,Casual,Movie,Simulation,Violent,...,Sexual Content,Animation & Modeling,Sports,Photo Editing,Web Publishing,Utilities,Massively Multiplayer,Early Access,Software Training,Game Development
0,579920,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,False
1,579930,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,579940,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,579960,False,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,579970,False,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,569685,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
939,569686,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
940,569687,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
941,569688,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [36]:
engine, conn = open_db_connection()
import sqlalchemy

def write_to_table(df, table_name, replace=False):
    # df.to_csv('placeholder.csv')

    dtypedict = sqlcol(df)
    # df = df.applymap(str)
    # print(df.head())

    if replace:
        if_exists = 'replace'
    else:
        if_exists = 'append'

    df.to_sql(
        name=table_name,
        con=engine,
        if_exists=if_exists,  # todo: this is not right, should be append, but leaving replace for testing - if i create
                              # tables with specific column names it should solve this
        index=False,
        dtype=dtypedict
    )
    
def sqlcol(dfparam):

    dtypedict = {}
    for i,j in zip(dfparam.columns, dfparam.dtypes):
        print(i)
        if "object" in str(j):
            dtypedict.update({i: TEXT})

        elif "datetime" in str(j):
            dtypedict.update({i: DateTime()})

        elif "float" in str(j):
            dtypedict.update({i: Float(precision=3, asdecimal=True)})

        elif "int" in str(j):
            dtypedict.update({i: Integer()})

        elif "bool" in str(j):
            dtypedict.update({i: Boolean()})

        else:
            dtypedict.update({i: NVARCHAR(length=255)})

    #todo: print this out for now so i can capture column names/types and build a constant definition
#     print(dtypedict)
    return dtypedict

In [ ]:
def update_table(data, table_name, replace_table=False):
    '''
    replace_table: if True, will replace all app_list table data, not update

    Pulls all app_ids from Steam, compares them to anything on the app_list table already and adds any new ones with a
    last_update of yesterday

    return nothing
    '''

    if engine.has_table(table_name) and not replace_table:
        appid_dict_list = pd.read_sql_table(
            table_name,
            con=engine,
            columns=[
                'app_id',
                'last_update'
            ],
            parse_dates=[
                'last_update'
            ],
        ).to_dict(orient='records')

    else:
        appid_dict_list = []

    yesterday = datetime.date.today() - datetime.timedelta(days=1)

    for app in data:
        appid = app['appid']

        if not any(d.get('app_id', None) == appid for d in appid_dict_list):
            appid_dict_list.append({'app_id': appid, 'last_update': yesterday})

    app_list_df = pd.DataFrame.from_records(appid_dict_list)
    if app_list_df.empty:
        print('app list dataframe is empty!  Probbably need a new steam key')
        raise SystemExit(0)

    app_list_df['last_update'] = pd.to_datetime(app_list_df['last_update'])

    app_list_df.to_sql(
        settings.APP_LIST_TABLE,
        engine,
        if_exists='replace',
        index=False,
        chunksize=10000,
        dtype={
            "app_id": Integer,
            "last_update": DateTime
        }
    )

In [77]:
def convert_col_to_bool_table(df, col, table_name, replace=False):
    '''
    Takes in a single columns in a dataframe, determines all unique values,
    creates a column for each unique value in the dataframe and fills it
    with a bool for each row indicating if that values exists for that row.

    Stores this as a tables with steam_appid

    assumes column splits out into ['id','description'] pairs for uniqueness


    df: Dataframe
    col: column to split out into multiple bool columns
    table_name: table to append new values
    replace: defaults False - indicates if a table should be replaced or appended
    Will use description to build new column name
    '''
    # first we need to create a table of all possible values then store those so we can access them
    # combine everything into single column

    new_df = df[['steam_appid', col]].copy()

    s = new_df[col].apply(pd.Series)
    num_cols = s.shape[1]

    # from pandas docs: Iteratively appending to a Series can be more computationally intensive than a single
    # concatenate. A better solution is to append values to a list and then concatenate the list with the original
    # Series all at once.
    listified = s[0].tolist()
    for x in range(1, num_cols):
        sub_list = s[x].dropna().tolist()
        listified += sub_list

    y = pd.Series(listified)

    y = y.dropna()

    # pull the dict out to columns
    z = y.apply(pd.Series)
    
    z = z.drop_duplicates(keep="first")
    
    # build df of current table columns
    if engine.has_table(table_name):
        boolean_df = pd.read_sql_table(
            table_name,
            con=engine,
        )
    
    combined_column_list = list(set().union(z.description.tolist(), boolean_df.columns.tolist()))

    # create a new column for each unique value
    for description in combined_column_list:
        new_df[description] = False

    # then fill those columns in the Dataframe with bools
    for index, row in df.iterrows():
        if type(row[col]) == float:
            continue
        for item in row[col]:
            # because you can't update on  iterrows()
            new_df.at[index, item['description']] = True

    # drop any rows in the existing data that are being updated by the new df
    boolean_df = boolean_df[~boolean_df['steam_appid'].isin(new_df['steam_appid'])]
    
    #append the old dataframe onto the new, filling in False for any extra columns
    df_append = new_df.append(boolean_df, verify_integrity=True).fillna(False)
    
    # drop the original column at the end of processing
    df_append.drop(axis=1, columns=col, inplace=True)
    df.drop(axis=1, columns=col, inplace=True)

    write_to_table(df_append, table_name, True)

    return df

In [78]:
df_orig = pd.DataFrame({'steam_appid': [111,2222,3333,123], 'A': [True, True, False, True], 'B': [False, True, True,True], 'C': [True, False, True, False]})
print(df_orig)
df_new = pd.DataFrame({'steam_appid': [555, 666, 123], 'A': [True, False, True], 'B': [True, True, True], 'D': [False, True, False]})
print(df_new)


df_append = df_new.append(df_orig, verify_integrity=True).fillna(False)
print(df_append)

   steam_appid      A      B      C
0          111   True  False   True
1         2222   True   True  False
2         3333  False   True   True
3          123   True   True  False
   steam_appid      A     B      D
0          555   True  True  False
1          666  False  True   True
2          123   True  True  False


ValueError: Indexes have overlapping values: Int64Index([0, 1, 2], dtype='int64')

In [79]:
df = load_data()
df_clean = convert_col_to_bool_table(df, 'genres', 'genre_table_testing')



     steam_appid  Photo Editing  Video Production  Strategy  Sexual Content  \
0              0              1                 0         0               0   
1              0              0                 0         0               0   
2              0              0                 0         1               0   
3              0              0                 0         0               0   
4              0              0                 0         0               0   
..           ...            ...               ...       ...             ...   
938            0              0                 0         0               0   
939            0              0                 0         0               0   
940            0              0                 0         0               0   
941            0              0                 0         0               0   
942            0              0                 0         0               0   

     Casual  Free to Play  Racing  Violent  Movie  

In [9]:
APP_LIST_TABLE = 'app_list'

engine, conn = open_db_connection()

def compile_new_applist():
    '''
    maybe this table should have a "processed" column and then i can just grab ids that have not been processed and
    don't have to compare anything - would require updates to the table and new records are stored to other tables
    '''

    ALL_APPS_URL = 'http://api.steampowered.com/ISteamApps/GetAppList/v0002/?key=E1C250C03DE1EE30169FCAC651DCB715&format=json'
    all_apps = get_request(ALL_APPS_URL)
    
    appid_dict = {}
    
    print(list(all_apps['applist']['apps'])[0:25])

    if engine.has_table(APP_LIST_TABLE):
        print('table found')
        appid_list = pd.read_sql(APP_LIST_TABLE).to_dict('records')
    else:
        appid_list = []
        
    print("list length (pre): " + str(len(appid_list)))

    yesterday = datetime.date.today() - datetime.timedelta(days=1)

    for app in all_apps['applist']['apps']:
        appid = app['appid']

        if appid not in appid_list:
            appid_dict['app_id'] = appid
            appid_dict['last_update'] = yesterday 
            appid_list.append(appid_dict)

    print("list length(post): " + str(len(appid_list)))
    
    app_list_df = pd.DataFrame.from_records(appid_list)

    print(app_list_df.shape)

    app_list_df.to_sql(
        APP_LIST_TABLE,
        engine,
        if_exists='replace',
        index=False,
        chunksize=500,
        dtype={
            "app_id": Integer,
            "last_update": DateTime
        }
    )
    return app_list_df

In [10]:
df = compile_new_applist()
df

[{'appid': 216938, 'name': 'Pieterw test app76 ( 216938 )'}, {'appid': 660010, 'name': 'test2'}, {'appid': 660130, 'name': 'test3'}, {'appid': 873710, 'name': 'Paris: Jigsaw Puzzles'}, {'appid': 873720, 'name': 'Pac Adventures 3D'}, {'appid': 873730, 'name': 'Under The War'}, {'appid': 873740, 'name': 'Tranquility Base Mining Colony: The Moon - Explorer Version'}, {'appid': 873790, 'name': 'Freebot : Battle for FreeWeb'}, {'appid': 873810, 'name': 'Cars Arena'}, {'appid': 873830, 'name': 'Subcube'}, {'appid': 873850, 'name': 'PASSAGE'}, {'appid': 873860, 'name': "Pop Island - Let's code !!! "}, {'appid': 873870, 'name': 'Sudden Strike 4 - Africa: Desert War'}, {'appid': 873871, 'name': 'Sudden Strike 4 - The Pacific War'}, {'appid': 873880, 'name': 'Railway Empire - The Great Lakes'}, {'appid': 873900, 'name': 'Lost Artifacts: Golden Island'}, {'appid': 873910, 'name': 'Rail World'}, {'appid': 873920, 'name': 'Among the Dead - Deluxe Edition'}, {'appid': 873930, 'name': 'ALIENS INVADED

TypeError: read_sql() missing 1 required positional argument: 'con'

In [ ]:
existing = [{'appid': 216938, 'name': 'Pieterw test app76 ( 216938 )'}, {'appid': 660010, 'name': 'test2'}, {'appid': 660130, 'name': 'test3'}, {'appid': 402590, 'name': 'Double Fine Adventure'}, {'appid': 402591, 'name': 'Double Fine Adventure: Ep02 - A Promise of Infinite Possibility'}, {'appid': 402592, 'name': 'Double Fine Adventure: Ep03 - Codename: Reds'}, {'appid': 402593, 'name': 'Double Fine Adventure: Ep04 - Walking Around In Our Drawings'}, {'appid': 402594, 'name': 'Double Fine Adventure: Ep05 - It’s Gonna Get Hairy'}, {'appid': 402595, 'name': 'Double Fine Adventure: Ep06 - That Bagel Filter Thing'}, {'appid': 402596, 'name': "Double Fine Adventure: Ep07 - We'll Handle It"}, {'appid': 402597, 'name': 'Double Fine Adventure: Ep08 - Adventure Games Are Not Dead'}, {'appid': 402598, 'name': 'Double Fine Adventure: Ep09 - Broken Age'}, {'appid': 402599, 'name': 'Double Fine Adventure: EP10 - Part One of Something Great'}, {'appid': 402600, 'name': 'Double Fine Adventure: EP11 - Ship It'}, {'appid': 402601, 'name': 'Double Fine Adventure: EP12 - A Whole Different Game Experience'}, {'appid': 402602, 'name': 'Double Fine Adventure: Ep13 - Crash Landing a Plane'}, {'appid': 402603, 'name': 'Double Fine Adventure: EP14 - I Think This is a Winner'}, {'appid': 402604, 'name': 'Double Fine Adventure: Ep15 - Evergreen Games'}, {'appid': 402605, 'name': "Double Fine Adventure: Ep16 - This Time it's Just for Love"}, {'appid': 402606, 'name': 'Double Fine Adventure: Ep17 - Deadline for Tim'}, {'appid': 402607, 'name': 'Double Fine Adventure: Ep18 - Constipation and Defcon 4'}, {'appid': 402608, 'name': 'Double Fine Adventure: EP19 - Last Call'}, {'appid': 402609, 'name': 'Double Fine Adventure: Ep20 - We Did Our Job'}, {'appid': 402620, 'name': 'Kindred Spirits on the Roof'}, {'appid': 402630, 'name': 'The Coma'}]

app_list = [{'appid': '123', 'date': "a"},{'appid': '1234', 'date': "a"},{'appid': '1235', 'date': "a"}]

for app in existing:
    appid = app['appid']

    if appid not in app_list:
        appid_dict['app_id'] = appid
        appid_dict['last_update'] = yesterday 
        appid_list.append(appid_dict)
            
            
            
            


In [86]:
def list_to_string(df, field):
    '''
    takes in a dataframe column that is a list and separates it into
    just the contents of the list, replacing the original columns.

    df: dataframe to alter
    field: column to remove list
    '''
    if df[field].dropna().empty:
        print(field + ' column was empty')
        return df

    df['liststring'] = [','.join(map(str, l)) for l in df[field]]

    df.drop(axis=1, columns=[field], inplace=True)
    df.rename(columns={'liststring': field}, inplace=True)

    return df

In [87]:
df = pd.DataFrame({'A': [[1,2],[1,2],[]], 'B': [1,2,3]})
df2 = list_to_string(df, 'A')
df2

,B,A
0,1,"1,2"
1,2,"1,2"
2,3,
